# Portfolio Trimming Strategy: Does It Beat Buy-and-Hold?
## Testing Systematic Profit-Taking on Index-Heavy Portfolios

**Research Period:** January 2015 - November 2024 (9.85 years)  
**Initial Capital:** $100,000  
**Data Source:** Yahoo Finance (real historical data)  
**Author:** Research Report  
**Date:** November 2024

## Executive Summary

This research tests whether systematic portfolio trimming—taking 20% profits at predetermined gain thresholds—beats buy-and-hold on a realistic 60/40 index/stock portfolio.

**Quick backstory:** Initial testing on individual stocks (AAPL, MSFT, NVDA, TSLA) showed trimming dramatically underperforming due to NVDA's 280x gain. However, this scenario is unrealistic—nobody buys NVDA at $0.48 and holds through a 280-bagger. I rebuilt the analysis with a portfolio an actual investor would own: 60% index funds (SPY, QQQ, VOO) and 40% blue-chip stocks.

**Main finding:** Trimming nearly matched buy-and-hold (21.4% vs 21.7% CAGR) while delivering materially better risk-adjusted returns.

**Key insight:** The viability of trimming depends entirely on portfolio composition. Index-heavy portfolios can trim successfully. Concentrated stock portfolios cannot.

**Bottom line:** For index-focused investors, trimming at +100-150% thresholds with pro-rata reinvestment offers similar returns with 6% less maximum drawdown. Worth considering if you value smoother portfolio rides over maximizing absolute returns.

**Critical limitation:** This analysis does NOT model taxes. In taxable accounts, trimming creates taxable events that could eliminate its viability. Trimming only makes sense in tax-deferred accounts (IRA, 401k).

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Methodology

### 1.1 Portfolio Construction

The portfolio represents a realistic diversified investor—not a concentrated stock picker who perfectly identifies every 100-bagger.

**60% Index Fund Allocation:**
- **SPY (S&P 500):** 30% allocation
- **QQQ (Nasdaq-100):** 20% allocation  
- **VOO (Vanguard S&P 500):** 10% allocation

**40% Blue-Chip Stock Allocation:**
- **AAPL (Apple):** 15% allocation
- **MSFT (Microsoft):** 15% allocation
- **TSLA (Tesla):** 10% allocation

This 60/40 split prevents any single position from dominating portfolio returns—the key difference from concentrated portfolios where one outlier invalidates trimming strategies.

### 1.2 Trimming Strategy Mechanics

**Core Logic:**
- When any position gains +X% from its cost basis → Sell 20% of shares
- Trim thresholds tested: +50%, +100%, +150%
- After trim: Cost basis resets to current price × 1.05

**Reinvestment Modes (4 variants per threshold):**

1. **Pro-rata:** Redistribute proceeds proportionally across all holdings (maintains portfolio allocation)
2. **SPY:** Invest all proceeds into S&P 500 index (safe haven approach)
3. **Cash:** Hold proceeds in cash (tests opportunity cost of sitting idle)
4. **Dip-buy-5%:** Wait for 5% S&P drop, then buy SPY/QQQ alternating (tactical timing attempt)

**Total strategies tested:** 13
- 1 buy-and-hold baseline
- 12 trimming variants (3 thresholds × 4 reinvestment modes)

### 1.3 Performance Metrics

**Primary Metrics:**
- **CAGR (Compound Annual Growth Rate):** Annualized return rate over full period
- **Sharpe Ratio:** Risk-adjusted return (higher = better return per unit of risk)
- **Maximum Drawdown:** Largest peak-to-trough portfolio decline
- **Volatility:** Annualized standard deviation of returns

**Secondary Metrics:**
- **Sortino Ratio:** Downside-focused risk adjustment
- **Total Return:** Cumulative percentage gain
- **Number of Trims:** Frequency of profit-taking events

### 1.4 Data Specifications

**Source:** Yahoo Finance historical data  
**Period:** January 1, 2015 - November 5, 2024  
**Trading Days:** 2,477 days (~9.85 years)  
**Initial Capital:** $100,000 allocated according to portfolio percentages  
**Assumptions:**
- No transaction costs (conservative for trimming strategies)
- No taxes modeled (critical limitation—see conclusions)
- No slippage or execution delays
- Dividends included in price data
- No portfolio rebalancing except via trims

In [ ]:
# Load results from index-focused backtest
results = pd.read_csv('results_index_focus/index_focus_results.csv', index_col=0)

# Display top 5 strategies by final value
print("Index-Focused Portfolio - Top 5 Strategies by Final Value")
print("=" * 80)
top5 = results.nlargest(5, 'final_value')[['final_value', 'cagr', 'sharpe_ratio', 'max_drawdown']]
top5.columns = ['Final Value ($)', 'CAGR', 'Sharpe Ratio', 'Max Drawdown']
display(top5)

## 2. Results

### 2.1 Overall Performance Summary

The table below shows the key performance metrics for all 13 strategies tested, sorted by final portfolio value.

In [ ]:
# Full results table with formatted columns
display_cols = ['final_value', 'cagr', 'sharpe_ratio', 'max_drawdown', 'volatility', 'num_trades']
results_display = results[display_cols].copy()
results_display.columns = ['Final Value ($)', 'CAGR', 'Sharpe', 'Max DD', 'Volatility', 'Trades']
results_display = results_display.sort_values('Final Value ($)', ascending=False)

print("Complete Strategy Rankings")
print("=" * 100)
display(results_display)

In [ ]:
# Display performance comparison visualization
display(Image('visualizations/phase3_performance_comparison.png'))

**Key Numbers:**

| Strategy | Final Value | CAGR | Sharpe | Max DD |
|----------|-------------|------|--------|--------|
| **Buy-and-Hold** | $688,711 | 21.69% | 0.90 | -46.3% |
| Trim@+150% (pro-rata) | $670,744 | 21.36% | 0.92 | -42.8% |
| Trim@+100% (pro-rata) | $670,503 | 21.36% | 0.94 | -40.8% |
| Trim@+50% (pro-rata) | $659,955 | 21.16% | 0.94 | -39.7% |

**Main Finding:** Buy-and-hold won, but barely—only 2.6% ahead of the best trimming strategy ($17,967 difference over 10 years on $100,000 initial capital).

**Translation:** You could have trimmed systematically and ended up with 97.4% of buy-and-hold returns while experiencing materially smoother portfolio volatility.

### 2.2 Risk-Adjusted Returns

While buy-and-hold achieved the highest absolute returns, trimming strategies delivered superior risk-adjusted performance.

In [ ]:
# Risk-return scatter plot
display(Image('visualizations/risk_return_scatter.png'))

In [ ]:
# Sharpe ratio comparison
display(Image('visualizations/sharpe_ratio_comparison.png'))

**Sharpe Ratio Improvements:**
- Buy-and-hold: **0.90 Sharpe ratio**
- Trim@+100% (pro-rata): **0.94 Sharpe** (+4.4% improvement)
- Trim@+50% (pro-rata): **0.94 Sharpe** (+4.4% improvement)

**What this means:** Trimming strategies delivered similar returns with measurably less volatility. For risk-averse investors, this is a meaningful improvement—you're getting paid similar returns for taking less risk.

### 2.3 Drawdown Analysis

Maximum drawdown measures the worst peak-to-trough decline an investor experienced. This is often the metric that determines whether an investor can psychologically stick with a strategy.

In [ ]:
# Drawdown comparison visualization
display(Image('visualizations/drawdown_comparison.png'))

**Maximum Drawdown Comparison:**
- Buy-and-hold: **-46.3%** max drawdown
- Trim@+150% (pro-rata): **-42.8%** (-3.5% less pain)
- Trim@+100% (pro-rata): **-40.8%** (-5.5% less pain)
- Trim@+50% (pro-rata): **-39.7%** (-6.6% less pain)

**Real-world impact:** If you had $500,000 invested, that's a $33,000 smaller drawdown. For many investors, experiencing $297,000 in losses instead of $330,000 is psychologically meaningful—even if the long-term CAGR is 0.3% lower.

The key question: Is 6% less maximum drawdown worth giving up 0.3% CAGR? That's a personal decision based on your risk tolerance.

### 2.4 The Dip-Buy Strategy: Perfect Execution, Suboptimal Results

One variant tested was: trim at gains, hold cash, wait for 5% S&P drops, then buy SPY/QQQ.

**Theoretical appeal:** Capture downside opportunities and buy at discounts.

**Actual results:** Underperformed immediate reinvestment despite perfect execution.

In [ ]:
# Dip-buy timeline visualization
display(Image('visualizations/dip_buy_timeline.png'))

In [ ]:
# Analyze dip-buy performance
dip_strategies = results[results.index.str.contains('dip-buy')].copy()
immediate_strategies = results[results.index.str.contains('pro-rata')].copy()

print("Dip-Buy vs Immediate Reinvestment Comparison")
print("=" * 80)
print("\nDip-Buy Strategies:")
display(dip_strategies[['final_value', 'cagr', 'sharpe_ratio', 'num_dip_buys', 'avg_dip_size']])
print("\nImmediate Reinvestment (Pro-Rata):")
display(immediate_strategies[['final_value', 'cagr', 'sharpe_ratio', 'num_trades']])

**Dip-Buy Results:**
- **Trim@+100% (dip-buy):** $621,394 final value (21 trades, 9 dips executed)
- **Trim@+100% (pro-rata):** $670,503 final value (14 trades, immediate reinvestment)
- **Difference:** $49,109 underperformance (-7.3%)

**Why did dip-buying underperform?**

Opportunity cost. The backtest executed 9 dips perfectly, averaging 7.67% drops captured. But while waiting for those dips, the market continued climbing. The opportunity cost of sitting in cash exceeded the benefit of buying at discounts.

**The lesson:** In bull markets, time in market > timing the market. Even perfect dip execution can't overcome missed upside.

### 2.5 Reinvestment Mode Rankings

Four reinvestment modes were tested for each trim threshold. Here's how they ranked:

In [ ]:
# Extract reinvestment mode and calculate averages
mode_results = results.copy()
mode_results['mode'] = mode_results.index.str.extract(r'\((.*)\)')[0]
mode_results.loc['Buy-and-Hold', 'mode'] = 'baseline'

mode_avg = mode_results.groupby('mode')[['final_value', 'cagr', 'sharpe_ratio', 'max_drawdown']].mean()
mode_avg = mode_avg.sort_values('final_value', ascending=False)
mode_avg.columns = ['Avg Final Value ($)', 'Avg CAGR', 'Avg Sharpe', 'Avg Max DD']

print("Reinvestment Mode Performance (Averaged Across All Thresholds)")
print("=" * 80)
display(mode_avg)

**Mode Rankings (Averaged):**

1. **Baseline (Buy-and-Hold):** $688,711 - Hard to beat the simple approach
2. **Pro-rata reinvestment:** $666,734 avg - Maintains exposure to best performers
3. **SPY reinvestment:** $647,049 avg - Safe but over-concentrates in S&P
4. **Dip-buy-5%:** $617,009 avg - Good risk metrics but opportunity cost hurts
5. **Cash holding:** $517,501 avg - Sitting idle is portfolio poison

**Key insight:** Pro-rata reinvestment won among trimming modes because it maintained diversified exposure while rebalancing. You captured gains without abandoning winning positions.

## 3. Analysis: Why Trimming Nearly Tied

### 3.1 Diversification Prevents Outlier Dominance

The critical difference between this portfolio and concentrated stock portfolios: **No single position could dominate returns.**

**Index fund performance (60% allocation):**
- SPY: +229% total return (steady climb)
- QQQ: +409% total return (tech-heavy growth)
- VOO: +229% total return (tracks SPY)

**Stock performance (40% allocation):**
- TSLA: +1,561% (10% allocation = 156% portfolio contribution)
- AAPL: +553% (15% allocation = 83% portfolio contribution)
- MSFT: +672% (15% allocation = 101% portfolio contribution)

**The key:** Even though TSLA gained 15x, it was only 10% of the portfolio. Trimming reduced it proportionally, but the index funds (60% allocation) provided steady baseline returns that couldn't be "mistakenly trimmed."

**Contrast with individual stock portfolios:** When NVDA gained 280x in our initial testing, it overwhelmed the portfolio. Trimming that position was catastrophically wrong. But in a 60/40 index/stock portfolio, no single winner can dominate enough to invalidate trimming.

### 3.2 The Risk-Return Trade-Off

Trimming offered a favorable trade-off for risk-averse investors:

**What you give up:**
- 0.3-1.0% CAGR (21.7% → 21.4% for best trimming strategy)
- $17,967 in final value ($688k → $670k)

**What you get:**
- 5-7% less maximum drawdown (-46% → -40%)
- 4% better Sharpe ratio (0.90 → 0.94)
- Behavioral benefit: Less panic-inducing volatility

**Is this trade worth it?** That's personal. But it's a reasonable trade for investors who:
- Value sleep-at-night portfolio stability
- Are closer to retirement (less time to recover from drawdowns)
- Have behavioral concerns about holding through 46% drawdowns
- Operate in tax-deferred accounts (no tax drag per trim)

### 3.3 Trim Threshold Analysis

Three thresholds were tested: +50%, +100%, +150%. Higher thresholds = fewer trims = better performance.

In [ ]:
# Trim frequency analysis
display(Image('visualizations/trim_frequency_analysis.png'))

In [ ]:
# Calculate threshold statistics
threshold_stats = []
for threshold in [50, 100, 150]:
    thresh_strategies = results[results.index.str.contains(f'+{threshold}%')]
    avg_trades = thresh_strategies['num_trades'].mean()
    avg_cagr = thresh_strategies['cagr'].mean()
    avg_sharpe = thresh_strategies['sharpe_ratio'].mean()
    threshold_stats.append({
        'Threshold': f'+{threshold}%',
        'Avg Trades': avg_trades,
        'Avg CAGR': avg_cagr,
        'Avg Sharpe': avg_sharpe
    })

threshold_df = pd.DataFrame(threshold_stats)
print("Trim Threshold Performance Summary")
print("=" * 60)
display(threshold_df)

**Findings:**
- **+50% threshold:** 23 avg trims → 21.16% avg CAGR (overtrimming penalty)
- **+100% threshold:** 14 avg trims → 21.36% avg CAGR (optimal balance)
- **+150% threshold:** 10 avg trims → 21.36% avg CAGR (nearly optimal)

**Sweet spot:** +100-150% thresholds. They trim frequently enough to provide risk reduction but not so frequently that they incur excessive opportunity cost.

**Avoid +50% threshold:** Too aggressive. You're trimming winners too early and missing significant upside. The 0.2% CAGR penalty proves overtrimming hurts returns without improving risk metrics meaningfully.

### 3.4 CAGR Comparison Across All Strategies

In [ ]:
# CAGR comparison visualization
display(Image('visualizations/cagr_comparison.png'))

## 4. Practical Guidance and Conclusions

### 4.1 When to Use Trimming Strategies

**Trimming works for:**

✅ **Index-heavy portfolios** (60%+ allocation to broad index funds)  
✅ **Risk-averse investors** who value smooth returns over maximum absolute performance  
✅ **Tax-deferred accounts** (IRA, 401k) where trims don't create immediate tax liabilities  
✅ **Behavioral guardrails** - Helps investors avoid panic selling by systematically taking gains  
✅ **Retirees or near-retirees** who can't afford large drawdowns and have shorter recovery horizons  

**Trimming does NOT work for:**

❌ **Concentrated stock portfolios** (<10 positions with high-conviction bets)  
❌ **High-conviction growth picks** with 10x-100x potential (you'll trim your winners too early)  
❌ **Taxable accounts** - Tax drag likely eliminates the strategy's viability (see section 4.3)  
❌ **Maximum return seekers** - Buy-and-hold won by 2.6%; if you want absolute max returns, don't trim  
❌ **Stock pickers identifying outliers** - If you can identify NVDA at $0.48, don't trim it  

### 4.2 Implementation Guidelines

If you decide trimming fits your profile, follow these guidelines:

**1. Use high thresholds:** +100% or +150%, NOT +50%
   - Lower thresholds overtrim and reduce returns without improving risk metrics
   - Sweet spot: Let winners run to 2x-2.5x before trimming

**2. Reinvest pro-rata:** Maintain proportional exposure across your portfolio
   - Don't concentrate reinvestment into a single position
   - Pro-rata keeps you diversified and exposed to future winners

**3. Don't wait for dips:** Immediate reinvestment beats market timing
   - Even perfect dip execution underperformed by 7%
   - Opportunity cost of sitting in cash exceeds dip discount benefits

**4. Check quarterly, not daily:** Set calendar reminders every 90 days
   - Reduces behavioral temptation to overtrade
   - Prevents emotional decision-making from short-term volatility

**5. Model taxes FIRST:** Run numbers for your specific tax situation
   - Use tax software to estimate actual post-tax returns
   - Compare trimming post-tax CAGR vs buy-and-hold post-tax CAGR
   - Only proceed if trimming still makes sense after taxes

### 4.3 The Tax Problem (Critical Limitation)

**This analysis did NOT model taxes.** This is a major limitation that likely eliminates trimming viability in taxable accounts.

**Tax implications:**
- Each trim creates a taxable event (capital gains on 20% of position)
- Long-term capital gains: 15-20% federal rate (plus state taxes)
- Short-term gains (if trimmed <1 year): Taxed as ordinary income (up to 37%)

**Estimated impact:**
- Trim@+100% (pro-rata): 14 trims over 10 years
- Assuming 15% LTCG rate and average $30k proceeds per trim
- Tax drag: ~$4,500 per trim × 14 = $63,000 in taxes paid
- Result: 21.4% CAGR → ~18.5% post-tax CAGR

**Comparison to buy-and-hold:**
- Buy-and-hold: No taxes until final sale
- 21.7% CAGR → ~20.8% post-tax CAGR (one tax event at end)

**Conclusion:** In taxable accounts, tax drag likely makes trimming unviable. Trimming only makes sense in tax-deferred accounts (IRA, 401k, 403b) where you can trade without triggering immediate taxes.

### 4.4 Research Limitations

**What this study did NOT test:**

1. **Bear market performance:** 2015-2024 was predominantly bullish. How would trimming perform in 2000-2010 sideways markets or 2008-style crashes?

2. **Different time periods:** Bull markets favor buy-and-hold. Sideways/volatile markets might favor trimming more heavily.

3. **Transaction costs:** Assumed commission-free trading. Even $5 per trade would reduce trimming returns.

4. **Alternative trim percentages:** Only tested 20% position reduction. Would 10%, 30%, or 50% trims perform better?

5. **Dynamic thresholds:** Tested fixed +50/100/150% thresholds. Would volatility-adjusted thresholds improve performance?

6. **Portfolio rebalancing:** Buy-and-hold drifted from initial 60/40 allocation. Would periodic rebalancing change outcomes?

7. **Dividend reinvestment:** Yahoo Finance data includes dividends in price. How would explicit dividend reinvestment affect results?

**Future research directions:**
- Test across multiple market regimes (bull, bear, sideways)
- Model tax implications explicitly for different account types
- Test dynamic threshold adjustment based on VIX or portfolio volatility
- Compare against periodic rebalancing strategies
- Analyze sector rotation with trimming strategies

### 4.5 Final Recommendation

**For most index investors: Stick with buy-and-hold.**

The 2.6% performance edge over 10 years ($688k vs $670k) isn't worth the added complexity, monitoring requirements, and potential tax implications. Simple strategies are often the best strategies.

**Exception: Tax-deferred accounts + risk-averse investors**

If you meet ALL these criteria, trimming is worth considering:
1. Your portfolio is in an IRA, 401k, or other tax-deferred account
2. You have 60%+ allocation to index funds (not concentrated stocks)
3. You value smoother portfolio rides more than maximizing absolute returns
4. You can stick to a disciplined quarterly review process
5. You're comfortable with 21.4% CAGR instead of 21.7% CAGR

**If that's you:** Use Trim@+100% with pro-rata reinvestment. You'll get 97.4% of buy-and-hold returns with materially better risk metrics.

**The key question to ask yourself:**

*Is 6% less maximum drawdown worth 0.3% less CAGR?*

- If yes → Trim at +100% thresholds
- If no → Buy and hold

**The simplest strategy is often the best strategy.** For most investors, that's buy-and-hold.

---

## Appendix: Complete Results Table

In [ ]:
# Display complete results sorted by final value
print("Complete Strategy Performance Results")
print("=" * 120)
complete_results = results.sort_values('final_value', ascending=False)
display(complete_results)

## Reproducibility

**Code location:** `/Users/austinwallace/sandbox/stock_strategies/trim_strat_test/`

**Key files:**
- `run_backtest_index_focus.py` - Backtest engine with all 13 strategies
- `generate_visualizations.py` - Chart generation code
- `manual_data/*.csv` - Historical price data from Yahoo Finance
- `results_index_focus/index_focus_results.csv` - Raw results

**To reproduce:**

```bash
# Run backtest
python run_backtest_index_focus.py

# Generate visualizations
python generate_visualizations.py
```

**Dependencies:**
- Python 3.8+
- pandas, numpy, matplotlib, seaborn
- See `requirements.txt` for complete list

**Data source:** Yahoo Finance historical data (2015-01-01 to 2024-11-05)

**Research date:** November 2024